# <center>Critical AI</center>
<center>ENGL 54.41</center>
<center>Dartmouth College</center>
<center>Winter 2026</center>
<pre>Created: 01/15/2026; Updated: 01/23/2026</pre>

In [ ]:
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

from gensim import matutils
import numpy as np
from numpy import dot
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.manifold import TSNE
import torch

from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
# Load Google News 200 Model (a much smaller model -- 50MB)
google_model = KeyedVectors.load_word2vec_format("../models/google-vectors.w2v",
                                                 binary = True)

In [ ]:
# "Interview" the model
vocab_size, dim = google_model.vectors.shape
print("vocab:", vocab_size)
print("embedding size:", dim)

## Clustering

We can automatically group embeddings together into clusters using an algorithm call k-Means. This finds clusters by partitioning data into _k_ clusters. We specify the number of clusters that we want prior to running the algorith. How to pick a value for _k_? Good question. We have to make a guess or know something about the data. It's just another sign of subjectivity and choice involved in machine learning.

In [ ]:
from sklearn.cluster import KMeans

# here we'll decide that we want fifty clusters of our vectors.
# vocab_size / 50 = means potentially many tokens/words per cluster but maybe more meaningful clusters?

kmeans = KMeans(n_clusters=50, random_state=42, n_init="auto")
kmeans.fit(google_model.vectors)

In [ ]:
# assign cluster labels to all vectors (to all vocabulary)
cluster_labels = kmeans.labels_

# these are center of the clusters--this will not correspond to a specific
# vector but the center of vector space comprised of vectors assigned to this
# cluster.
centroids = kmeans.cluster_centers_

In [ ]:
# let's visualize these centroids and label with the cluster id
plt.scatter(
    centroids[:, 0],
    centroids[:, 1],
    marker="o",       # circles are nice
    s=6,              # size of the marker
    linewidths=3,     # increase size
    color="g",        # green
)

# add labels to the cluster points
for i in range(centroids.shape[0]):
     plt.annotate(i, xy = (centroids[:, 0][i], centroids[:, 1][i]), xytext = (3, 3),
        textcoords = 'offset points', ha = 'left', va = 'top')
plt.title("k-Means Clusters of Vector Data")
plt.xticks(())
plt.yticks(())
plt.show()

## Interrogating a Cluster

We can explore centroids that are close together: these might have similar
semantic meanings even if the vocabulary are in distinct clusters.

In [ ]:
# This one looks promising. The centroids, as mentioned above, contain vector values. 
# They will be the same size as our embedding space:
centroids[27].shape

In [ ]:
# We can query the model for vectors most similar to this imaginary location in vector space:
google_model.most_similar(positive=[centroids[27]], topn=10)

In [ ]:
# Let's select another centroid located close to that one: 
google_model.most_similar(positive=[centroids[16]], topn=10)

In [ ]:
# And another:
google_model.most_similar(positive=[centroids[46]], topn=10)

In [ ]:
# This one looks like a real outlier--why?
google_model.most_similar(positive=[centroids[26]], topn=10)

In [ ]:
# Try some other clusters!

## Plotting Neighbors in Vector Space

We've already seen [PCA](https://en.wikipedia.org/wiki/Principal_component_analysis) (principal components analysis) as a technique of dimensionality reduction to reduce our high dimension (vector width of 200) data to two dimensions (x,y) for plotting; now we'll use another technique to do the same. This one is thought to work especially well with this sort of data. It is called t-distributed stochastic neighbor embedding or [t-SNE](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding) and can make useful 2D and 3D renderings of our data.

In [ ]:
def plot_neighbors(term):
    if term in google_model:
        vocab = [v[0] for v in google_model.most_similar(term,topn=50)]
        embs = np.array([google_model[v] for v in vocab])
        tsne = TSNE(n_components=2, perplexity=2, max_iter=1000, random_state=42)
        embeddings_2d = tsne.fit_transform(torch.tensor(embs))
        xs, ys = embeddings_2d[:, 0], embeddings_2d[:, 1]
        plt.figure(figsize=(8, 6))
        scatter = plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], alpha=0.7)
        for i, w in enumerate(vocab):
          plt.annotate(w, xy = (xs[i], ys[i]), xytext = (3, 3),
                       textcoords = 'offset points', ha = 'left', va = 'top')
        plt.title(f't-sne plot of neighbors of {term}')
        plt.grid(True)
        plt.xticks(())
        plt.yticks(())
        plt.tight_layout()
        plt.show()
    else:
        print(f'{term} not found in model vocab')

In [ ]:
plot_neighbors("blizzard")

In [ ]:
# to which cluster has that token been assigned?
cluster_labels[google_model.index_to_key.index("blizzard")]

In [ ]:
# what's that cluster look like?
google_model.most_similar(positive=[centroids[18]], topn=10)

In [ ]:
# now try some more!